# 4. Shell and Variables: Environment Variables, Local Variables, Global Variables, Special Variables, Arrays, Reserved Words

><span style='color:green'>**[TIP]** It is possible to execute two commands one after another on a single line. To do this, separate them with a semicolon `;`.</span>

In [1]:
echo "first command" > file1.dat; wc file1.dat

       1       2      14 file1.dat


In this section, we will describe various types of variables that exist in the `bash` shell. They are incredibly helpful in writing scripts and performing advanced operations. It is important to familiarize yourself with the terminology describing different variables and understand the concept of invoking a subshell to fully grasp the material presented below. While working in the `bash` shell, there is no obstacle to invoking another available shell. At that moment, a subordinate shell, called a __subshell__, will be created under our main shell. To check the available shells in our system, we can list the contents of the `shells` file located in the `/etc` directory.

In [2]:
cat /etc/shells

# List of acceptable shells for chpass(1).
# Ftpd will not allow users to connect who are not using
# one of these shells.

## /opt/local/bin/bash
/usr/local/bin/bash
/bin/bash
/bin/csh
/bin/dash
/bin/ksh
/bin/sh
/bin/tcsh
/bin/zsh


## 4.1 Environment/Program/Local/Global Variables

In the previous section, we presented a very simple script written in the `bash` shell. We also introduced the concept of variables - called **program variables**, which are declared by the user. In the script, once a variable is created, you can refer to it using the `$` symbol. In the bash environment, there is also the concept of **environment variables**, which modify and define our own environment. Examples of such variables include:

- `$HOME` -> displays the path to your home directory.
- `$USER` -> displays the username.
- `$SHELL` -> displays the currently used shell in the terminal.
- `$PWD` -> displays the path to the current directory.

Here's an example of displaying an environment variable:

In [7]:
echo "My username is: '"$USER"' and I'm using shell located at: "$SHELL

My username is: 'krzkot' and I'm using shell located at: /usr/local/bin/bash/


Variables can be **local** or **global**. Local variables are only visible in the shell where they were created, while global variables are visible in every subshell. To make our variable (whether it's a newly created program variable or a modified environment variable) a global variable, we need to assign it the appropriate attribute. This can be done using the `export` command. The `printenv` command is used to display all available environment variables.

Here's an example of exporting a variable:

In [10]:
myvar="local_var"; echo $myvar

local_var


At the moment, the variable `$myvar` is local. I can invoke another subshell (`bash`) in my terminal and check if it is visible there.

```bash
krzkot@krzkot ~ bash                     # subshell bash
krzkot@krzkot ~ echo $myvar              # subshell "cannot see" this variable

krzkot@krzkot ~ exit                     # exit from subshell
krzkot@krzkot ~ echo $myvar              # main shell still "sees" the variable
local_var
krzkot@krzkot ~ export myvar             # let's export the variable
krzkot@krzkot ~ echo $myvar              # main shell still "sees" the variable
local_var
krzkot@krzkot ~ bash                     # subshell bash
krzkot@krzkot ~ echo $myvar.             # subshell also "sees" the variable which is now global
local_var
```

## 4.2 Special variables

Special variables are available to the user for reading only. It means that it's not possible to change their values. These are selected characters and numbers used for convenient script handling. The digits **0-9** handle the arguments passed to our script.
- The variable `$0` displays the name of the current shell or the name of the executed script (we can say it's an argument with index 0).

In [11]:
echo $0

/usr/local/bin/bash


The shell name has been displayed. Let's modify our script `tbase.bash` so that it displays the variable `$0` at the beginning.

 ```bash
#!/bin/bash

echo $0   # Let's add the line to display the special variable

start=$(head -n 2 files/TIC102090493-s2-121s_lc3_fppt.dat | tail -n 1)
start=$(echo $start | cut -d " " -f 1)
end=$(tail -n 1 files/TIC102090493-s2-121s_lc3_fppt.dat)
end=$(echo $end | cut -d " " -f 1)
tbase=$(echo $end-$start | bc) # timebase calculation
echo $tbase
```

In [14]:
bash scripts/tbase_mod.bash

scripts/tbase_mod.bash
27.4044052


As expected, the script name was displayed. Now let's see what the variables with other digits are used for.

- Variables in the range `$0-9` will contain the arguments passed to our script on the command line. Arguments are provided after the script name, separated by spaces.

We need to modify our script again (`tbase_arg.bash`) to use the variables `$0-9`. Let's start with a simple display using the `echo` command.

 ```bash
#!/bin/bash

echo $1 $2   # display the special variables (first two arguments)

start=$(head -n 2 TIC102090493-s2-121s_lc3_fppt.dat | tail -n 1)
start=$(echo $start | cut -d " " -f 1)
end=$(tail -n 1 TIC102090493-s2-121s_lc3_fppt.dat)
end=$(echo $end | cut -d " " -f 1)
tbase=$(echo $end-$start | bc)
echo $tbase
```

In [17]:
bash scripts/tbase_arg.bash first second third

first second
27.4044052


As you can see, the first two arguments, `pierwszy` and `drugi`, were displayed. The third argument, `trzeci`, although provided, was not displayed because there was no command in the script to print it to the screen. However, let's try to use argument capturing for something more useful. Notice that currently our script only performs the operation for a file named `TIC102090493-s2-121s_lc3_fppt.dat`. We can pass the file name as an argument to our script but it needs to be with a relative directory. The modification to our script will be as follows:

 ```bash
#!/bin/bash
start=$(head -n 2 $1 | tail -n 1) # # instead of filename, we provide $1
start=$(echo $start | cut -d " " -f 1)
end=$(tail -n 1 $1)
end=$(echo $end | cut -d " " -f 1)
tbase=$(echo $end-$start | bc)
echo $tbase
```

In [20]:
bash scripts/tbase_name.bash files/TIC102090493-s2-121s_lc3_fppt.dat

27.4044052


In [21]:
bash scripts/tbase_name.bash files/TIC12974182-s2-121s_lc3_fppt.dat

12.0541511


Our script now executes if we provide the file name as the first argument for calculating the time base. If we don't provide an argument, our script will hang, and if we provide the name of a non-existent file, an error will be raised. In this example, a file named `somefile.dat`, which does not exist, is passed as an argument.

In [24]:
bash scripts/tbase_name.bash jakisplik.dat

head: jakisplik.dat: No such file or directory
tail: jakisplik.dat: No such file or directory

Parse error: bad expression
    <stdin>:1




 - `$@` taka zmienna wyświetli wszystkie podane argumenty
 - `$$` numer procesu aktualnej powłoki (wszystki procesy uruchomione w danym terminalu można podjerzeć za pomocą komendy `ps`)

In [25]:
ps; echo; echo 'Current shell process number is: '$$

  PID TTY           TIME CMD
58824 ttys000    0:00.22 -bash -l
59319 ttys000    0:33.21 /Users/krzkot/opt/anaconda3/bin/python /Users/krzkot/o
62817 ttys001    0:00.08 /usr/local/bin/bash --rcfile /Users/krzkot/opt/anacond
61529 ttys003    0:00.16 -zsh

Current shell process number is: 62817


><span style='color:green'>**[TIP]** The process number can be used to terminate a specific process (using the `kill` command).</span>

## 4.3 Array Variables

In `bash`, you can also create an array or, more precisely, a variable that will store an array. An array is a collection of values that you want to preserve, separated by spaces. Each element of the array will have an index assigned to it, starting from 0. The array has no size limit.

To refer to an array, you use the index enclosed in curly braces `${}`. Here's an example of creating and accessing an array:

```bash
my_array=("value1" "value2" "value3")
echo ${my_array[0]}   # Output: value1
echo ${my_array[1]}   # Output: value2
echo ${my_array[2]}   # Output: value3
```

You can access individual elements of the array using their respective indexes. **Remember that the first value has an index of 0.**

Arrays in `bash` provide a convenient way to store and manipulate collections of values. They are useful when you need to work with multiple related values within a single variable.

In [27]:
my_array=(one two three four five six)

In [29]:
echo ${my_array[0]}; echo ${my_array[3]}

one
four


If you don't specify an index, the first element (index 0) will be displayed. Additionally, when working with array variables, you can:

- Retrieve all elements: Instead of specifying an index, you can use `*` or `@`.
- Get the length of an array element: Precede the variable name with `#`.

Here's an example that demonstrates these concepts:

```bash
my_array=("value1" "value2" "value3")
echo ${my_array[*]}   # Output: value1 value2 value3
echo ${my_array[@]}   # Output: value1 value2 value3
echo ${#my_array[1]}  # Output: 6 (length of "value2")
echo ${#my_array[*]}  # Output: 3 (number of elements in the array)
echo ${#my_array[@]}  # Output: 3 (number of elements in the array)
```

><span style='color:green'>**[TIP] Using `*` or `@` as the index allows you to retrieve all elements of the array. Prepending `#` to the variable name gives you the length of a specific element or the number of elements in the entire array.**</span>

This can be useful when you need to perform operations or calculations based on the contents of an array.

In [30]:
echo "All array elements: "${my_array[*]}

All array elements: one two three four five six


In [34]:
echo "Number of characters in second element: ${#my_array[1]}"

Number of characters in second element: 3


In [35]:
echo "Number of elements in array:" ${#my_array[*]}

Number of elements in array: 6


You can also add an element to an array with any index (you don't have to add them sequentially). In this example, we will add an element with index 10. Elements with indices 6-9 will not exist.

In [43]:
my_array[10]="eleven"

In [45]:
echo "Element with index 10: "${my_array[10]}; echo "All array elements: "${my_array[*]}

Element with index 10: eleven
All array elements: one two three four five six eleven


To remove an element at a specific index from an array, you can use the `unset` command. It it also used to remove entire variable.

In [47]:
echo "Element with index 4: "${my_array[4]}

Element with index 4: five


In [48]:
unset my_array[4]

In [49]:
echo "Element with index 4: "${my_array[4]}; echo "All array elements: "${my_array[*]}

Element with index 4: 
All array elements: one two three four six eleven


As can be seen, the element with index 4 does not exist, and when displaying all elements, the name "five" does not appear. Similarly, to remove the entire array, you need to specify `*` or `@` as the index.

In [53]:
unset my_array[*]

In [54]:
echo ${my_array[*]}

## 4.4 Reserved words

Reserved words have special meaning in the `bash` shell and should not be used as variable names. Their list is provided below:

- `if`
- `then`
- `else`
- `elif`
- `fi`
- `case`
- `esac`
- `for`
- `while`
- `until`
- `do`
- `done`
- `in`
- `function`
- `select`
- `time`
- `return`
- `exit`
- `break`
- `continue`
- `declare`
- `readonly`
- `local`
- `export`
- `eval`
- `set`
- `unset`
- `shift`
- `source`
- `trap`
- `exec`
- `true`
- `false`
- `null`

It is advisable to avoid using these words as variable names to prevent conflicts with the shell's syntax and functionality.